# AtomsBase integration

[AtomsBase.jl](https://github.com/JuliaMolSim/AtomsBase.jl) is a common interface
for representing atomic structures in Julia. DFTK directly supports using such
structures to run a calculation as is demonstrated here.

In [1]:
using DFTK
using AtomsBase
using Unitful
using UnitfulAtomic

# Construct a system in the AtomsBase world
a = 10.26u"bohr"  # Silicon lattice constant
lattice = a / 2 * [[0, 1, 1.],  # Lattice as vector of vectors
                   [1, 0, 1.],
                   [1, 1, 0.]]
atoms  = [:Si => ones(3)/8, :Si => -ones(3)/8]
system = periodic_system(atoms, lattice; fractional=true)

FlexibleSystem(Si₂, AtomsBase.Periodic, box=[[0.0, 5.13, 5.13], [5.13, 0.0, 5.13], [5.13, 5.13, 0.0]]u"a₀")

System is an AtomsBase-compatible system. To use it in DFTK,
we attach pseudopotentials, construct a DFT model, discretise and solve:

In [2]:
system = attach_psp(system; family="hgh", functional="lda")

model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.921693749820                   -0.69    5.2
  2   -7.926219381310       -2.34       -1.22    1.0
  3   -7.926857635025       -3.20       -2.35    3.2
  4   -7.926864962455       -5.14       -3.09    3.2
  5   -7.926865047257       -7.07       -3.40    2.4
  6   -7.926865092919       -7.34       -4.44    4.0
  7   -7.926865092856   +  -10.20       -4.80    1.5


At any point we can also get back the DFTK model as an
AtomsBase-compatible `AbstractSystem`:

In [3]:
newsystem = atomic_system(model)

FlexibleSystem(Si₂, AtomsBase.Periodic, box=[[0.0, 5.13, 5.13], [5.13, 0.0, 5.13], [5.13, 5.13, 0.0]]u"a₀")